In [1]:
from nilmtk import DataSet
from nilmtk.utils import print_dict

import pandas as pd
import numpy as np

import datetime as dt

In [2]:
ds = DataSet("./AMPds2.h5")

In [3]:
ds.buildings

OrderedDict([(1, Building(instance=1, dataset='AMPds'))])

In [4]:
bld = ds.buildings[1]
bld

Building(instance=1, dataset='AMPds')

In [5]:
elecs = bld.elec
elecs

MeterGroup(meters=
  ElecMeter(instance=1, building=1, dataset='AMPds', site_meter, appliances=[])
  ElecMeter(instance=2, building=1, dataset='AMPds', appliances=[Appliance(type='light', instance=1)])
  ElecMeter(instance=3, building=1, dataset='AMPds', appliances=[Appliance(type='light', instance=2)])
  ElecMeter(instance=4, building=1, dataset='AMPds', appliances=[Appliance(type='light', instance=3)])
  ElecMeter(instance=5, building=1, dataset='AMPds', appliances=[Appliance(type='unknown', instance=1)])
  ElecMeter(instance=6, building=1, dataset='AMPds', appliances=[Appliance(type='unknown', instance=2)])
  ElecMeter(instance=7, building=1, dataset='AMPds', appliances=[Appliance(type='sockets', instance=1)])
  ElecMeter(instance=8, building=1, dataset='AMPds', appliances=[Appliance(type='unknown', instance=3)])
  ElecMeter(instance=9, building=1, dataset='AMPds', appliances=[Appliance(type='unknown', instance=4)])
  ElecMeter(instance=10, building=1, dataset='AMPds', appliances=[A

In [6]:
meters = elecs.meters
meters

[ElecMeter(instance=1, building=1, dataset='AMPds', site_meter, appliances=[]),
 ElecMeter(instance=2, building=1, dataset='AMPds', appliances=[Appliance(type='light', instance=1)]),
 ElecMeter(instance=3, building=1, dataset='AMPds', appliances=[Appliance(type='light', instance=2)]),
 ElecMeter(instance=4, building=1, dataset='AMPds', appliances=[Appliance(type='light', instance=3)]),
 ElecMeter(instance=5, building=1, dataset='AMPds', appliances=[Appliance(type='unknown', instance=1)]),
 ElecMeter(instance=6, building=1, dataset='AMPds', appliances=[Appliance(type='unknown', instance=2)]),
 ElecMeter(instance=7, building=1, dataset='AMPds', appliances=[Appliance(type='sockets', instance=1)]),
 ElecMeter(instance=8, building=1, dataset='AMPds', appliances=[Appliance(type='unknown', instance=3)]),
 ElecMeter(instance=9, building=1, dataset='AMPds', appliances=[Appliance(type='unknown', instance=4)]),
 ElecMeter(instance=10, building=1, dataset='AMPds', appliances=[Appliance(type='unkno

In [7]:
df_list = []

for meter in meters:
    
    # get appliance name/id
    # handle main error
    main_id = 1
    try:
        appliance_name = meter.appliances[0].identifier.type
        appliance_id = meter.appliances[0].identifier.instance
    except:
        appliance_name = "main"
        appliance_id = main_id
        main_id += 1
    
    # load data frame
    df = next(meter.load())
    
    # flatten multi-level column
    df.columns = ['_'.join(col).strip() for col in df.columns.values]
    df = df.reset_index()
    
    df['appliances_type'] = appliance_name
    df['appliances_id'] = appliance_id
    df['appliances_name'] = f"{appliance_name} {appliance_id}"
    
    # datetime
    df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'], utc=True)
    df['TIMESTAMP'] = df['TIMESTAMP'].dt.tz_convert(None)
    df['TIMESTAMP'] = df['TIMESTAMP'] + dt.timedelta(hours=-7)
    
    df_list.append(df)
    
    df.to_csv(f"./data/raw_appliances/{appliance_name}_{appliance_id}.csv", index=False)
    
    print(f"{appliance_name} {appliance_id} \n {len(df)}")

main 1 
 1051200
light 1 
 1051200
light 2 
 1051200
light 3 
 1051200
unknown 1 
 1051200
unknown 2 
 1051200
sockets 1 
 1051200
unknown 3 
 1051200
unknown 4 
 1051200
unknown 5 
 1051200
fridge 1 
 1051200
unknown 6 
 1051200
unknown 7 
 1051200
heat pump 1 
 1051200
unknown 8 
 1051200
light 4 
 1051200
sockets 2 
 1051200
unknown 9 
 1051200
television 1 
 1051200
sockets 3 
 1051200
electric oven 1 
 1051200


In [8]:
pd.concat(df_list).to_csv("./data/AMPDs2.csv", index=False)